# Download data

In [1]:
import os
import zipfile
import kaggle
# use kaggle api to download the data or manully download the data urself
## ! Kaggle competitions download -c "jane-street-real-time-market-data-forecasting" 

# make the directory to be "data/jane-street-real-time-market-data-forecasting" and extract file there u f the code dont work do it manually
## os.makedirs("data/jane-street-real-time-market-data-forecasting", exist_ok=True) # create directory for jane-street-real-time-market-data-forecasting-housing
## !unzip jane-street-real-time-market-data-forecasting.zip -d data/jane-street-real-time-market-data-forecasting/

# Kaggle Competition 

In [2]:
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import r2_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler


In [3]:
# function for import the data .parquet to pandas dataframe
def download_data(i): # the data have 10 partiton 0 to 9
    file_path = f"data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    df = pd.read_parquet(file_path)
    print(f"import train data part {i}")
    return df

In [4]:
# test/example of the import fuction
# train0 = download_data(0)
train9 = download_data(9) # there are just more data in the train.parquet partition 9

import train data part 9


In [5]:
# import test data 
file_path  = f"data/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"
test = pd.read_parquet(file_path)

# Data Preprocessing

## Data cleaning

In [6]:
# feature selection ## select every feature except 0-4 
feature_cols = [f"feature_{idx:02d}" for idx in range(79)]
train9[feature_cols]

#select target values
target_cols = ["responder_6"]
train9[target_cols]

#select lag values
time_cols = ["date_id","time_id"]
train9[time_cols]

# select the weight values
weight_cols = ["weight"]


## drop the other responder column

In [7]:
drop_col = [f"responder_{idx}"for idx in range(6)] + [f"responder_{idx}"for idx in range(7,9)]
train9 = train9.drop(columns= drop_col , axis= 1, errors= 'ignore')

# split the data and transfrom them into ts 

In [9]:
# remove the nan values
def replace_nan_with_mean_or_zero(column):
    mean_value = column.mean()
    if pd.notnull(mean_value):
        return column.fillna(mean_value)
    else:
        return column.fillna(0)